In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-08-02


Found rundate 2024-08-01 10:00:03 < 2024-08-02 at commit 872fd4b1; returning commit 54d8fdd


'2024-08-02 10:00:01'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-08-02 10:00:01, NJSP was reporting 356 YTD deaths
Current YTD Deaths (2025-08-02 10:00:01-04:00): 324
Previous year YTD Deaths (adjusted; 2024-08-02 10:00:01): 355.3101514537758
Projected 2025 total: 636.7060894549587
-8.8% change


607ce7fd5ac48f48ff1a6ca52f535b0d1fdccf08: FAUQStats cache miss: 2024, Fri Aug 02 10:00:01 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,East Oak St,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,27,State Highway 27,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13323,01,Atlantic,0108,Egg Harbor Twsp,40,State Highway 40,2.0,2.0,0.0,0.0,0.0,NaN,NaN,2024-07-26 20:18:00-04:00
13322,04,Camden,0412,Cherry Hill Twsp,38,State Highway 38,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-07-26 23:11:00-04:00
13325,09,Hudson,0904,Harrison Town,280,Interstate 280 MP 15.3,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-07-27 01:24:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,14,2,6,0,21
Bergen,12,1,5,0,18
Burlington,14,4,3,0,20
Camden,12,2,8,1,22
Cape May,5,1,0,0,6
Cumberland,6,2,3,0,10
Essex,6,3,14,1,24
Gloucester,11,4,2,0,14
Hudson,3,1,3,1,7


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

58.5% through the year, 41.5% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,18,31,14
Bergen,driver,11,14,12
Burlington,driver,19,31,14
Camden,driver,12,19,12
Cape May,driver,2,8,5
...,...,...,...,...
Salem,crashes,8,14,10
Somerset,crashes,9,18,7
Sussex,crashes,7,12,3


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,40,0,25,4,13
Bergen,33,0,15,2,13
Burlington,36,0,24,4,12
Camden,32,1,19,3,11
Cape May,21,0,16,1,0
Cumberland,23,0,15,6,8
Essex,45,1,13,10,26
Gloucester,40,0,26,14,5
Hudson,20,1,7,1,18


In [9]:
projected.to_csv(PROJECTED_CSV)